In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df2 = pd.read_csv('7_dwarfs_train.csv')
df2['is_7_dwarfs'] = 1

In [3]:
df3 = pd.read_csv('alien_saucers.csv')
df3['is_alien_saucers'] = 1

In [4]:
df4 = pd.read_csv('dinosaur.csv')
df4['is_dino'] = 1

In [5]:
df5 = pd.read_csv('expedition_everest.csv')
df5['is_exp_everest'] = 1

In [6]:
df6 = pd.read_csv('flight_of_passage.csv')
df6['is_flight_pass'] = 1

In [7]:
df7 = pd.read_csv('kilimanjaro_safaris.csv')
df7['is_kil_safaris'] = 1

In [8]:
df8 = pd.read_csv('navi_river.csv')
df8['is_navi_river'] = 1

In [9]:
df9 = pd.read_csv('pirates_of_caribbean.csv')
df9['is_pirates'] = 1

In [10]:
df10 = pd.read_csv('rock_n_rollercoaster.csv')
df10['is_rocknroll'] = 1

In [11]:
df11 = pd.read_csv('slinky_dog.csv')
df11['is_slinky'] = 1

In [12]:
df12 = pd.read_csv('soarin.csv')
df12['is_soarin'] = 1

In [13]:
df13 = pd.read_csv('spaceship_earth.csv')
df13['is_spaceship_earth'] = 1

In [14]:
df14 = pd.read_csv('splash_mountain.csv')
df14['is_splash_mntn'] = 1

In [15]:
df15 = pd.read_csv('toy_story_mania.csv')
df15['is_toy_story'] = 1

In [16]:
df = pd.concat([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15])

In [17]:
df

,date,datetime,SACTMIN,SPOSTMIN,is_7_dwarfs,is_alien_saucers,is_dino,is_exp_everest,is_flight_pass,is_kil_safaris,is_navi_river,is_pirates,is_rocknroll,is_slinky,is_soarin,is_spaceship_earth,is_splash_mntn,is_toy_story
0,01/01/2015,2015-01-01 07:51:12,NaN,45.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2015,2015-01-01 08:02:13,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2015,2015-01-01 08:05:30,54.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2015,2015-01-01 08:09:12,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2015,2015-01-01 08:16:12,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217122,12/31/2019,2019-12-31 23:30:02,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
217123,12/31/2019,2019-12-31 23:37:02,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
217124,12/31/2019,2019-12-31 23:44:02,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
217125,12/31/2019,2019-12-31 23:51:02,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [18]:
df['SPOSTMIN'] = df['SPOSTMIN'].fillna((df['SPOSTMIN'].median()))
df['is_7_dwarfs'] = df['is_7_dwarfs'].fillna(0)
df['is_alien_saucers'] = df['is_alien_saucers'].fillna(0)
df['is_dino'] = df['is_dino'].fillna(0)
df['is_exp_everest'] = df['is_exp_everest'].fillna(0)
df['is_flight_pass'] = df['is_flight_pass'].fillna(0)
df['is_kil_safaris'] = df['is_kil_safaris'].fillna(0)
df['is_navi_river'] = df['is_navi_river'].fillna(0)
df['is_pirates'] = df['is_pirates'].fillna(0)
df['is_rocknroll'] = df['is_rocknroll'].fillna(0)
df['is_slinky'] = df['is_slinky'].fillna(0)
df['is_soarin'] = df['is_soarin'].fillna(0)
df['is_spaceship_earth'] = df['is_spaceship_earth'].fillna(0)
df['is_splash_mntn'] = df['is_splash_mntn'].fillna(0)
df['is_toy_story'] = df['is_alien_saucers'].fillna(0)
df['is_alien_saucers'] = df['is_toy_story'].fillna(0)

In [19]:
df

,date,datetime,SACTMIN,SPOSTMIN,is_7_dwarfs,is_alien_saucers,is_dino,is_exp_everest,is_flight_pass,is_kil_safaris,is_navi_river,is_pirates,is_rocknroll,is_slinky,is_soarin,is_spaceship_earth,is_splash_mntn,is_toy_story
0,01/01/2015,2015-01-01 07:51:12,NaN,45.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01/01/2015,2015-01-01 08:02:13,NaN,60.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01/01/2015,2015-01-01 08:05:30,54.0,40.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01/01/2015,2015-01-01 08:09:12,NaN,60.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01/01/2015,2015-01-01 08:16:12,NaN,60.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217122,12/31/2019,2019-12-31 23:30:02,NaN,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217123,12/31/2019,2019-12-31 23:37:02,NaN,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217124,12/31/2019,2019-12-31 23:44:02,NaN,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217125,12/31/2019,2019-12-31 23:51:02,NaN,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
def drop_nulls(df, prop_required_column = .70, prop_required_row = .70):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [21]:
df = drop_nulls(df, prop_required_column = .70, prop_required_row = .70)

In [23]:
df = df.rename(columns = {'SPOSTMIN' : 'wait_time'})

In [24]:
df = df[df.wait_time > 0]

In [25]:
df.isnull().sum()

date                  0
datetime              0
wait_time             0
is_7_dwarfs           0
is_alien_saucers      0
is_dino               0
is_exp_everest        0
is_flight_pass        0
is_kil_safaris        0
is_navi_river         0
is_pirates            0
is_rocknroll          0
is_slinky             0
is_soarin             0
is_spaceship_earth    0
is_splash_mntn        0
is_toy_story          0
dtype: int64